In [9]:
from vedo import dataurl, Mesh, Plotter, Volume, settings, LinearTransform, Box, Line
import numpy as np
import pandas as pd
from normalize import normalize_shape,get_eigenvectors
import os
from pathlib import Path
import matplotlib.pyplot as plt
settings.default_backend = 'k3d'
np.set_printoptions(suppress=True)


In [18]:

def normalize_position(mesh:Mesh,inplace=True):
    """
    Shifts input mesh so that its center of mass coincides with the origin
    ----------------------------
    Args:
        Vedo Mesh
    Returns:
        Vedo Mesh
    """
    if(not inplace):
        wMesh=mesh.copy()
    else:
        wMesh=mesh
    LT = LinearTransform()
    LT.translate(wMesh.transform.position-wMesh.center_of_mass())
    LT.move(wMesh)
    return wMesh

def normalize_pose(mesh:Mesh,inplace=True):
    """
    Rotates mesh so that its major axes are aligned with the unit axes. 
    ----------------------------
    Input:
        Vedo Mesh
    Returns:
        Vedo Mesh
    """
    if(not inplace):
        wMesh=mesh.copy()
    else:
        wMesh=mesh
    eigenvectors,eigenvalues = get_eigenvectors(wMesh)
    ranking = np.argpartition(eigenvalues, 2)
    aligned_matrix = [eigenvectors[ranking[2]],
                      eigenvectors[ranking[1]],
                      eigenvectors[ranking[0]]]
    wMesh.vertices = np.dot(wMesh.vertices-mesh.center_of_mass(),np.transpose(aligned_matrix))
    return wMesh
    verts = []
    c = mesh.center_of_mass()
    for i in range(0, len(mesh.vertices)):
        v = mesh.vertices[i]
        p1 = np.dot(v-c, eigenvectors[ranking[2]])
        p2 = np.dot(v-c, eigenvectors[ranking[1]])
        p3 = np.dot(v-c, eigenvectors[ranking[0]])
        verts.append([p1, p2, p3])
    wMesh.vertices = verts
    return wMesh



In [19]:
#Rich Display

z_axis = Line([0,0,0], [0,0,2], lw=3).c("Blue")
y_axis = Line([0,0,0], [0,2,0], lw=3).c("Green")
x_axis = Line([0,0,0], [2,0,0], lw=3).c("Red")
unit_box = Box(width=1,height=1,length=1).c("Black").wireframe(True)

filepath = "../shapes\Door\D01104.obj"
mesh = Mesh(filepath).c("violet").flat()

print(mesh)
# Set the backcolor of the mesh to violet
# and show edges with a linewidth of 2

mesh2 = normalize_position(mesh,inplace=False).c("black")

print(mesh2)
mesh3 = normalize_pose(mesh2,inplace=False).c("Green")

eigenvectors, eigenvals = get_eigenvectors(mesh3)

print(mesh3)
print("EIGENVECTORS:",eigenvectors)

plot = Plotter(bg='white')
plot.show(mesh,mesh2,mesh3,x_axis,y_axis,z_axis,unit_box)

vedo.mesh.Mesh at (0x21ef8f432d0)                                          
name          : Mesh
file name     : ../shapes\Door\D01104.obj
color         : violet, rgb=(0.933, 0.510, 0.933), alpha=1.0
elements      : vertices=871 polygons=1,772 lines=0
position      : (0, 0, 0)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.276694, diagonal=1.05662
center of mass: (0.331757, 0.647215, 0.0456782)
bounds        : x=(0.0250, 0.486), y=(0.0250, 0.975), z=(0.0250, 0.0670)
celldata      : "GroupIds" (float32), dim=1, range=(0, 0)
vedo.mesh.Mesh at (0x21ef8f40ad0)                                          
name          : Mesh
file name     : ../shapes\Door\D01104.obj
color         : black, rgb=(0, 0, 0), alpha=1.0
elements      : vertices=871 polygons=1,772 lines=0
position      : (-0.331757, -0.647215, -0.0456782)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.276694, diagonal=1.05662
center of mass: (-2.14921e-9, 1.19757e-9, 8.75162e-11)
bounds  

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…